In [17]:
#  Loading All Required Frameworks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn import metrics
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from pandas.io.json import json_normalize

In [11]:
tweets_json = pd.read_json("tweets_DM.json",lines=True)
tweets = json_normalize(data = tweets_json['_source'])
data_identification = pd.read_csv("data_identification.csv")
emotion = pd.read_csv("emotion.csv")

In [12]:
tweets=tweets.rename(columns={"tweet.text":"tweet", "tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags"})
tweets = pd.merge(tweets, data_identification, on='tweet_id')

In [14]:
training_set = tweets[tweets['identification'] == 'train']
training_set = pd.merge(training_set, emotion, on='tweet_id')
training_set.drop(columns=['identification'], inplace=True)

testing_set = tweets[tweets['identification'] == 'test']
testing_set['emotion'] = ''
testing_set.drop(columns=['identification'], inplace=True)

/Users/bijonsetyawan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/bijonsetyawan/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [15]:
from nltk.tokenize import RegexpTokenizer

tknzr = RegexpTokenizer(r'\w+')
for index, row in training_set.iterrows():
  temp = tknzr.tokenize(row['tweet'])
  row['tweet'] = ' '.join(temp)
  
for index, row in testing_set.iterrows():
  temp = tknzr.tokenize(row['tweet'])
  row['tweet'] = ' '.join(temp)

In [18]:
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
tfidf = TfidfVectorizer(max_features=100000, stop_words='english',tokenizer=tokenizer.tokenize)

tfidf.fit(training_set['tweet'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x1a2cfe9a20>>,
        use_idf=True, vocabulary=None)

In [19]:
# transforming training sets
X_train = tfidf.transform(training_set['tweet'])
X_test = tfidf.transform(testing_set['tweet'])
y_train = training_set['emotion']
y_test = testing_set['emotion']

In [20]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=6, n_jobs=-1, max_iter=1000, multi_class="multinomial", solver='lbfgs', random_state=0)
clf.fit(X_train,y_train)
prediction = clf.predict(X_test)
testing_set['emotion'] = prediction

/Users/bijonsetyawan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [21]:
testing_set.drop(columns=['hashtags', 'tweet'], inplace=True)
testing_set = testing_set.rename(columns={'tweet_id': 'id'})
testing_set.to_csv("testing_result_100000.csv", index=False)

/Users/bijonsetyawan/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
testing_set

,id,emotion
2,0x28b412,trust
4,0x2de201,anticipation
9,0x218443,joy
30,0x2939d5,joy
33,0x26289a,trust
35,0x31c6e0,joy
37,0x32edee,disgust
46,0x3714ee,joy
49,0x235628,disgust
56,0x283024,joy
